## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [7]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

## Import the image from the source folder

In [10]:
import os
import pprint
test_images=os.listdir('./test_images')
image=test_images[3]
img=cv2.imread('./test_images/'+image)
gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


# # Calibrate and undistort the image 

In [11]:
def cal_undistort_image(img, objPts = objpoints, imgPts = imgpoints):
    """
    returns the undistorted image
    """
    ret,camera_matrix,distortion_coefficients,rotational_vectors,tangential_vectors=cv2.calibrateCamera(objPts,imgPts,gray_img.shape[0:2],None,None)
    undistort_img = cv2.undistort(img,camera_matrix,distortion_coefficients,None,camera_matrix)
    return undistort_img

def hls_to_s(img):
    """
    returns a 2D image ie the s channel of the HLS image
    """
    hls=cv2.cvtColor(img,cv2.COLOR_BGR2HLS) 
    mask=np.zeros_like(hls[:,:,0])
    #thresholding the pixel values of the mask
    mask[(hls[:,:,2] > 100) & (hls[:,:,0]<100)]=1
    s_channel=mask
    return s_channel

def absolute_sobel(img,orientation='x',sobel_kernel=3,thresh=(20,100)):
    """
    returns a binary thresholded image with sobel gradients
    """
    #taking sobel gradient 
    if orientation=='x':
        sobel_img=cv2.Sobel(img,cv2.CV_64F,1,0,ksize=sobel_kernel)
    elif orientation=='y':
        sobel_img=cv2.Sobel(img,cv2.CV_64F,0,1,ksize=sobel_kernel)
    #finding the absolute value     
    abs_sobel=np.absolute(sobel_img) 
    #scaling the pixel values to 8 bit
    scaled_sobel_img=np.uint8(255*abs_sobel/np.max(abs_sobel))
    #create a binary image with zero values
    binary_threshold=np.zeros_like(scaled_sobel_img)
    # change the zero values to ones where thresholding condition is met
    binary_threshold[(scaled_sobel_img >= thresh[0]) & (scaled_sobel_img <= thresh[1])]=1
    return binary_threshold

def mag_thresh(img,sobel_kernel=5,thresh=(30,100)):
    """
    This returns the binary thresholded image with both x and y sobel gradients
    """
    #taking sobel x gradient
    sobelx=cv2.Sobel(img,cv2.CV_64F,1,0,ksize=sobel_kernel)
    #taking sobel y gradient
    sobely=cv2.Sobel(img,cv2.CV_64F,0,1,ksize=sobel_kernel)
    #calculating the gradient magnitude
    abs_mag_sobel=np.absolute(np.sqrt(sobelx**2 + sobely**2))
    #scaling the pixel values to 8 bit
    scaled_sobel=np.uint8(255*abs_mag_sobel/np.max(abs_mag_sobel))
    #create a binary image with zero values
    binary_threshold=np.zeros_like(scaled_sobel)
    # change the zero values to ones where thresholding condition is met
    binary_threshold[(scaled_sobel >= thresh[0]) & (scaled_sobel <=thresh[1])]=1
    return binary_threshold

def dir_thresh(img,sobel_kernel=5,thresh=(0.7,1.3)):
    """
    This returns the binary image with thresholded angled sobel gradients
    """
    #taking sobel x gradient
    sobelx=cv2.Sobel(img,cv2.CV_64F,1,0,ksize=sobel_kernel)
    #taking sobel y gradient
    sobely=cv2.Sobel(img,cv2.CV_64F,0,1,ksize=sobel_kernel)
    #getting the sobel gradients based on the direction
    dir_grad=np.arctan2(np.absolute(sobely),np.absolute(sobelx))
    #create a binary image with zero values
    binary_threshold=np.zeros_like(dir_grad)
    # change the zero values to ones where thresholding condition is met
    binary_threshold[(dir_grad >= thresh[0]) & (dir_grad <=thresh[1] )]=1
    return binary_threshold

def perform_perspective_transform(img,img_size):
    """
    This returns the warped image ie a bird's eye view 
    """
    #the source coordinates on the image to be taken in consideration for a transform
    src = np.array([[585, 460], [203, 720], [1127, 720], [695, 460]]).astype(np.float32)
    #the sestination coordinates on the image to be taken in consideration for warping
    dst = np.array([[320, 0], [320, 720], [960, 720], [960, 0]]).astype(np.float32)
    #returns a transform matrix M
    M=cv2.getPerspectiveTransform(src,dst)
    #a warped image is returned
    warped=cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    return warped
 
def perform_inverse_perspective_transform(img,img_size):
    """
    This returns the original image from the transformed image
    """
    src = np.array([[585, 460], [203, 720], [1127, 720], [695, 460]]).astype(np.float32)
    dst = np.array([[320, 0], [320, 720], [960, 720], [960, 0]]).astype(np.float32)
    M=cv2.getPerspectiveTransform(dst,src)
    #the original image is returned
    original=cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    return original    


def hls_sobel_mask(img):
    """
     Applies the HLS and sobel masks to the image
    """
    #take a copy of the image
    img = img.copy()
    #convert image to grayscale
    gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #convert image to HLS
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # Apply a mask on HLS colour channels
    mask = np.zeros_like(hls[:, :, 0])
    # This selects pixels with higher than 100 saturation and lower than 100 hue
    mask[((hls[:, :, 2] > 100) & (hls[:, :, 0] < 100) ) ] = 1
    # Apply a sobel magnitude threshold
    # I apply a more lenient mag_thresh to the upper part of the transformed image, as this part is blurrier
    # and will therefore have smoother gradients.
    # On the bottom half, this selects pixels with >10 sobel magnitude, and on the top half, 
    # selects pixels with >35 sobel magnitude
    upper_mag = mag_thresh(gray_img, 3, (10, 255))
    lower_mag = mag_thresh(gray_img, 3, (35, 255))
    
    mag_mask = np.zeros_like(lower_mag)
    mag_mask[:int(mag_mask.shape[0]/2), :] = upper_mag[:int(mag_mask.shape[0]/2), :]
    mag_mask[int(mag_mask.shape[0]/2):, :] = lower_mag[int(mag_mask.shape[0]/2):, :]
    
    # Use the bitwise OR mask of both masks for the final mask
    final_mask = np.maximum(mag_mask, mask)

    # Return the transformed mask
    return final_mask                        

def find_peaks(final_mask):
    """
    Returns the inices of the left and right lanes
    """
    #finding the size of the image
    shape=final_mask.shape
    #taking the bottom section of image under consideration
    bottom_section=final_mask[-int(shape[0]/2):,]
    
    
    #the left peak ie the indices or x position of left lane
    left_peak=bottom_section[:,:int(shape[1]/2)].sum(axis=0).argmax()
    #the right peak ie the indices or the x position of right lane
    right_peak=bottom_section[:,int(shape[1]/2):].sum(axis=0).argmax() + int(shape[1]/2)

    return left_peak,right_peak
                        
def window_search(final_mask,left_peak,right_peak,no_of_strips=10,margin=80):
    """
    This applies the sliding window approach to find lane pixels, and then fits a polynomial to the found pixels.
    """
    #creating an array of windows or strips ie dividing the image into vertical windows or strips
    strips=[]
    assert final_mask.shape[0] % no_of_strips==0 , 'No of strips should be a factor of height of the image ie vertical resolution'
    #size or width of the strip
    width_of_strip=final_mask.shape[0]/no_of_strips
    for i in range(no_of_strips):
        strip=final_mask[i*width_of_strip:(i+1)*width_of_strip,:]
        strips.append(strip)
    # reverse theorder of strips ie start from the bottom to top        
    strips=strips[::-1]
    #store x positions of left lane 
    lefts=[left_peak]
    #store x positions of right lane 
    rights=[right_peak]
    left_px=[]
    left_py=[]
    right_px=[]
    right_py =[]
    for i ,strip in enumerate(strips):
        offset=(no_of_strips -i-1)*width_of_strip
        last_left=int(lefts[-1])
        last_right=int(rights[-1])
        # Only consider pixels within +-leeway of last strip location
        temp_left_strip=strip.copy()
        temp_left_strip[:, :last_left-margin]=0
        temp_left_strip[:,last_left+margin:]=0
        
        temp_right_strip=strip.copy()
        temp_right_strip[:,:last_right-margin]=0
        temp_right_strip[:,last_right+margin:]=0
        # Save the x, y pixel indexes for calculating the polynomial
        left_px.append(temp_left_strip.nonzero()[1])
        left_py.append(temp_left_strip.nonzero()[0] + offset)
        
        right_px.append(temp_right_strip.nonzero()[1])
        right_py.append(temp_right_strip.nonzero()[0] + offset)
    # Create x and y indice arrays for both lines
    left_px=np.concatenate(left_px)
    left_py=np.concatenate(left_py)
    right_px=np.concatenate(right_px)
    right_py=np.concatenate(right_py)
    # Fit the polynomials!
    left_poly=np.polyfit(left_py,left_px,2)
    right_poly=np.polyfit(right_py,right_px,2)
    
    return left_poly,right_poly

def plot_polygon(img_original,img_size,left_poly,right_poly):
    """
    Plot the polygon on the images
    """
    plot_y=np.linspace(0,img_original.shape[0]-1,img_original.shape[0])
    left_fit=left_poly[0]*plot_y**2 + left_poly[1]*plot_y + left_poly[2]
    right_fit=right_poly[0]*plot_y**2 +right_poly[1]*plot_y + right_poly[2]
    
    pts_left=np.array([np.transpose(np.vstack([left_fit,plot_y]))])
    pts_right=np.array([np.flipud(np.transpose(np.vstack([right_fit,plot_y])))])
    pts=np.hstack((pts_left,pts_right))
    # Create an overlay from the lane lines
    overlay_mask=np.zeros_like(img_original).astype(np.uint8)
   
    cv2.fillPoly(overlay_mask,np.int_([pts]),(0,255,0))
    # Apply inverse transform to the overlay to plot it on the original road
    overlay_mask=perform_inverse_perspective_transform(overlay_mask,img_size)
    
    return cv2.addWeighted(img_original,1,overlay_mask,0.3,0)

def find_curvature(poly,final_mask):
    yscale = 30 / 720 # Real world metres per y pixel
    xscale = 3.7 / 700 # Real world metres per x pixel
    
    # Convert polynomial to set of points for refitting
    ploty = np.linspace(0, final_mask.shape[0]-1, final_mask.shape[0])
    fitx = poly[0] * ploty ** 2 + poly[1] * ploty + poly[2]
    
    # Fit new polynomial
    fit_cr = np.polyfit(ploty * yscale, fitx * xscale, 2)
    
    # Calculate curve radius
    curverad = ((1 + (2 * fit_cr[0] * np.max(ploty) * yscale + fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * fit_cr[0])
    return curverad

def find_offset(l_poly,r_poly):
    #assuming camera is installed at the center of the car dashboard
    lane_width=3.7 #in metres
    h=720
    w=1280
    bottom_point_left=l_poly[0]*h**2 + l_poly[1]*h + l_poly[2]
    bottom_point_right=r_poly[0]*h**2 + r_poly[1]*h + r_poly[2]
    
    # no of pixels per metre
    meter_scale=lane_width/np.absolute(bottom_point_right -bottom_point_left)
    
    #midpoint in between  the lanes
    midpoint=np.mean([bottom_point_left,bottom_point_right])
    
    #offset between the camera and the lane center
    
    offset=(w/2 - midpoint) * meter_scale
    
    return offset


  

In [12]:
last_rad=None
last_left_poly=None
last_right_poly=None
def process_frame(img):
    global last_rad,last_left_poly,last_right_poly
    
    #weights for smoothing
    rad_factor=0.03
    poly_factor=0.3
    
    #undistorting the frame image
    undistort=cal_undistort_image(img) 
   
    #create image copy
    orig_img=undistort.copy()
    
    img_size=(undistort.shape[1],undistort.shape[0])
    #perform warping of the image
    warped=perform_perspective_transform(undistort,img_size)
    
    combined_binary_thresh =hls_sobel_mask(warped)
    
    #finding the lane lines x positions
    l,r=find_peaks(combined_binary_thresh)#
    #finding the coefficeints of polynomials fitting the lane lines
    l_poly,r_poly = window_search(combined_binary_thresh,l,r)
    
    
    if last_left_poly is None:
        last_left_poly=l_poly
        last_right_poly=r_poly
    else:
        l_poly=(1-poly_factor)* last_left_poly + (poly_factor) * l_poly
        r_poly=(1-poly_factor)*last_right_poly +(poly_factor) * r_poly
        last_left_poly=l_poly
        last_right_poly=r_poly
    
    #finding the curvature of the road
    
    l_rad=find_curvature(l_poly,combined_binary_thresh)
    r_rad=find_curvature(r_poly,combined_binary_thresh)
    rad=np.mean([l_rad,r_rad])
    
    if last_rad is None:
        last_rad=rad
    else:
        rad=(1-rad_factor) *rad + rad_factor * last_rad
    #plot the polygon
    result=plot_polygon(orig_img, (combined_binary_thresh.shape[1],combined_binary_thresh.shape[0]),l_poly,r_poly)
    
     # Write radius on image
    cv2.putText(result, 'Lane Radius: {}m'.format(int(last_rad)), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, 255)
    offset=find_offset(l_poly,r_poly)
    
    #Write lane offset on frame
    cv2.putText(result,'Lane Offset: {}m'.format(int(offset)),(10,100),cv2.FONT_HERSHEY_SIMPLEX,1.5,255)
    
    return result


In [13]:
process_frame(img)

C:\Users\Me\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:158: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


array([[[194, 158, 112],
        [194, 158, 112],
        [194, 158, 112],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[193, 157, 111],
        [194, 158, 112],
        [194, 158, 112],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[193, 157, 111],
        [194, 158, 112],
        [194, 158, 112],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ..., 
       [[ 78,  86, 106],
        [ 84,  92, 112],
        [ 88,  97, 117],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 79,  88, 108],
        [ 82,  91, 111],
        [ 86,  95, 115],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 80,  89, 109],
        [ 79,  88, 108],
        [ 79,  88, 108],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,

In [ ]:
from moviepy.editor import VideoFileClip

white_output = 'project_video_run4.mp4'
clip1 = VideoFileClip('./Input Videos/project_video.mp4')
white_clip = clip1.fl_image(process_frame) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_video_run4.mp4
[MoviePy] Writing video project_video_run4.mp4



 12%|████▌                                  | 149/1261 [01:03<08:05,  2.29it/s]